In [94]:
import socket

In [2]:
help(socket.socket)

Help on class socket in module socket:

class socket(_socket.socket)
 |  A subclass of _socket.socket adding the makefile() method.
 |  
 |  Method resolution order:
 |      socket
 |      _socket.socket
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, *args)
 |  
 |  __getstate__(self)
 |  
 |  __init__(self, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_STREAM: 1>, proto=0, fileno=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Wrap __repr__() to reveal the real class name and socket
 |      address(es).
 |  
 |  accept(self)
 |      accept() -> (socket object, address info)
 |      
 |      Wait for an incoming connection.  Return a new socket
 |      representing the connection, and the address of the client.
 |      For IP sockets, the address info is a pair (hostaddr, port).
 |  
 |  close(self)
 |      close()
 |      
 |      Close the socket.  It canno

In [3]:
so = socket.socket()

In [4]:
so

<socket.socket fd=55, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('0.0.0.0', 0)>

In [5]:
help(so)

Help on socket in module socket object:

class socket(_socket.socket)
 |  A subclass of _socket.socket adding the makefile() method.
 |  
 |  Method resolution order:
 |      socket
 |      _socket.socket
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, *args)
 |  
 |  __getstate__(self)
 |  
 |  __init__(self, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_STREAM: 1>, proto=0, fileno=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Wrap __repr__() to reveal the real class name and socket
 |      address(es).
 |  
 |  accept(self)
 |      accept() -> (socket object, address info)
 |      
 |      Wait for an incoming connection.  Return a new socket
 |      representing the connection, and the address of the client.
 |      For IP sockets, the address info is a pair (hostaddr, port).
 |  
 |  close(self)
 |      close()
 |      
 |      Close the socket.  It cann

In [6]:
so.bind(('127.0.0.1', 1234))

In [7]:
so.listen()

In [8]:
help(so.accept)

Help on method accept in module socket:

accept() method of socket.socket instance
    accept() -> (socket object, address info)
    
    Wait for an incoming connection.  Return a new socket
    representing the connection, and the address of the client.
    For IP sockets, the address info is a pair (hostaddr, port).



In [9]:
s, info = so.accept()

In [10]:
s

<socket.socket fd=56, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 1234), raddr=('127.0.0.1', 54028)>

In [11]:
info

('127.0.0.1', 54028)

In [12]:
s.recv(1024)

b'1234\r\n'

In [13]:
s.recv(1024)

b'abcd\r\n'

In [14]:
s.recv(1024)

b'qwer\r\n'

- 初始化socket 实例
- bind
- listen
- accept

In [16]:
s.send(b'1234')

4

In [17]:
s.send(b'miracle')

7

In [18]:
s.close()

In [19]:
so.close()

In [20]:
so = socket.socket()

In [21]:
so.bind(('127.0.0.1', 1234))

In [22]:
so.listen()

In [23]:
s, _ = so.accept()

In [24]:
s.recv(1024)

b'asdf\r\n'

In [25]:
s.send(b'miracle')

7

In [26]:
s.send(b'young')

5

In [27]:
s2, _ = so.accept()

In [28]:
s2.send(b's2')

2

In [29]:
s.close()

In [33]:
import threading

In [30]:
s2.close()

In [59]:
class ChatServer:
    def __init__(self, ip='127.0.0.1', port=1234):
        self.addr = (ip, port)
        self.sock = socket.socket()
        self.event = threading.Event()
        self.clients = {}
    
    def recv(self, so, ip, port):
        while not self.event.is_set():
            data = so.recv(1024).decode()
            if data.strip() == '/quit':
                so.close()
                self.clients.pop((ip, port))
                return
            for s in self.clients.values():
                s.send('{}:{}\n{}'.format(ip, port, data).encode())
    
    def accept(self):
        while not self.event.is_set():
            so, (ip, port) = self.sock.accept()
            self.clients[(ip, port)] = so
            threading.Thread(target=self.recv, name='client-{}:{}'.format(ip, port), args=(so, ip, port)).start()
    
    def start(self):
        # TODO 开启服务端
        # 是否需要启用多线程呢？
        self.sock.bind(self.addr)
        self.sock.listen()

        threading.Thread(target=self.accept, name='listen').start()        
        
    def stop(self):
        for so in self.clients.values():
            so.close()
        self.sock.close()
        self.event.set()

In [60]:
chat_server = ChatServer(port=1239)

In [61]:
chat_server.start()

In [62]:
chat_server.stop()

Exception in thread listen:
Traceback (most recent call last):
  File "/Users/miracleYoung/.pyenv/versions/3.6.1/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/miracleYoung/.pyenv/versions/3.6.1/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-59-c8298e251039>", line 20, in accept
    so, (ip, port) = self.sock.accept()
  File "/Users/miracleYoung/.pyenv/versions/3.6.1/lib/python3.6/socket.py", line 205, in accept
    fd, addr = self._accept()
ConnectionAbortedError: [Errno 53] Software caused connection abort
Exception in thread client-127.0.0.1:55142:
Traceback (most recent call last):
  File "/Users/miracleYoung/.pyenv/versions/3.6.1/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/miracleYoung/.pyenv/versions/3.6.1/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-59-c8298

In [63]:
so = socket.socket()
so.bind(('127.0.0.1', 10086))
so.listen()

In [64]:
s, _ = so.accept()

In [65]:
help(s.recv)

Help on built-in function recv:

recv(...) method of socket.socket instance
    recv(buffersize[, flags]) -> data
    
    Receive up to buffersize bytes from the socket.  For the optional flags
    argument, see the Unix manual.  When no data is available, block until
    at least one byte is available or until the remote end is closed.  When
    the remote end is closed and all data is read, return the empty string.



In [66]:
help(s.recv_into)

Help on built-in function recv_into:

recv_into(...) method of socket.socket instance
    recv_into(buffer, [nbytes[, flags]]) -> nbytes_read
    
    A version of recv() that stores its data into a buffer rather than creating 
    a new string.  Receive up to buffersize bytes from the socket.  If buffersize 
    is not specified (or 0), receive up to the size available in the given buffer.
    
    See recv() for documentation about the flags.



In [67]:
help(s.recvfrom)

Help on built-in function recvfrom:

recvfrom(...) method of socket.socket instance
    recvfrom(buffersize[, flags]) -> (data, address info)
    
    Like recv(buffersize, flags) but also return the sender's address info.



In [68]:
help(s.recvfrom_into)

Help on built-in function recvfrom_into:

recvfrom_into(...) method of socket.socket instance
    recvfrom_into(buffer[, nbytes[, flags]]) -> (nbytes, address info)
    
    Like recv_into(buffer[, nbytes[, flags]]) but also return the sender's address info.



In [69]:
help(s.recvmsg)

Help on built-in function recvmsg:

recvmsg(...) method of socket.socket instance
    recvmsg(bufsize[, ancbufsize[, flags]]) -> (data, ancdata, msg_flags, address)
    
    Receive normal data (up to bufsize bytes) and ancillary data from the
    socket.  The ancbufsize argument sets the size in bytes of the
    internal buffer used to receive the ancillary data; it defaults to 0,
    meaning that no ancillary data will be received.  Appropriate buffer
    sizes for ancillary data can be calculated using CMSG_SPACE() or
    CMSG_LEN(), and items which do not fit into the buffer might be
    truncated or discarded.  The flags argument defaults to 0 and has the
    same meaning as for recv().
    
    The return value is a 4-tuple: (data, ancdata, msg_flags, address).
    The data item is a bytes object holding the non-ancillary data
    received.  The ancdata item is a list of zero or more tuples
    (cmsg_level, cmsg_type, cmsg_data) representing the ancillary data
    (control messag

In [70]:
help(s.send)

Help on built-in function send:

send(...) method of socket.socket instance
    send(data[, flags]) -> count
    
    Send a data string to the socket.  For the optional flags
    argument, see the Unix manual.  Return the number of bytes
    sent; this may be less than len(data) if the network is busy.



In [71]:
help(s.sendall)

Help on built-in function sendall:

sendall(...) method of socket.socket instance
    sendall(data[, flags])
    
    Send a data string to the socket.  For the optional flags
    argument, see the Unix manual.  This calls send() repeatedly
    until all data is sent.  If an error occurs, it's impossible
    to tell how much data has been sent.



In [72]:
f = s.makefile()

In [73]:
f.read(5)

'sadfs'

In [74]:
w = s.makefile('w')

In [75]:
w.write('123456\n')

7

In [76]:
w.flush()

In [77]:
b = s.makefile('br')

In [78]:
b.readline()

b'sdfsdfsdfsdfsdf\r\n'

In [79]:
s.close()

In [80]:
b.close()

In [81]:
f.close()

In [82]:
w.close()

In [83]:
s.close()

In [84]:
so.close()

In [85]:
so = socket.socket()

In [86]:
so.getsockname()

('0.0.0.0', 0)

In [87]:
so.bind(('127.0.0.1', 10087))

In [88]:
so.getsockname()

('127.0.0.1', 10087)

In [91]:
so.getsockopt()

TypeError: getsockopt() takes at least 2 arguments (0 given)

In [92]:
so.timeout

In [93]:
so.settimeout = 3

AttributeError: 'socket' object attribute 'settimeout' is read-only

In [95]:
class ChatServer:
    def __init__(self, ip='127.0.0.1', port=1234):
        self.addr = (ip, port)
        self.sock = socket.socket()
        self.event = threading.Event()
        self.clients = {}
    
    def recv(self, so, ip, port):
        while not self.event.is_set():
            data = so.recv(1024).decode()
            if data.strip() == '/quit':
                so.close()
                self.clients.pop((ip, port))
                return
            for s in self.clients.values():
                s.send('{}:{}\n{}'.format(ip, port, data).encode())
    
    def accept(self):
        while not self.event.is_set():
            so, (ip, port) = self.sock.accept()
            self.clients[(ip, port)] = so
            threading.Thread(target=self.recv, name='client-{}:{}'.format(ip, port), args=(so, ip, port)).start()
    
    def start(self):
        # TODO 开启服务端
        # 是否需要启用多线程呢？
        self.sock.bind(self.addr)
        self.sock.listen()

        t = threading.Thread(target=self.accept, name='listen', daemon=True)
        try:
            t.start()
            t.join()
        except KeyboardInterrupt:
            self.stop()
        
    def stop(self):
        for so in self.clients.values():
            so.close()
        self.sock.close()
        self.event.set()

## UDP Server

In [107]:
so = socket.socket(type=socket.SOCK_DGRAM)

In [108]:
so.bind(('127.0.0.1', 1234))

In [98]:
so.accept()

OSError: [Errno 102] Operation not supported on socket

In [99]:
so.listen()

OSError: [Errno 102] Operation not supported on socket

In [100]:
so.recv(1024)

b'hello\n'

- 初始化 socket 实例，注意type 类型（socket.SOCK_DGRAM）
- bind

In [101]:
so.recv(1024)

b'hello\n'

In [102]:
so.recv(1024)

b'hello\n'

In [104]:
so.send(b'sdfsfd')

OSError: [Errno 39] Destination address required

- 如何知道对端地址
- 如何发送数据回去

In [109]:
so.sendto(b'miracle\n', ('127.0.0.1', 65432))

8

In [110]:
help(so.recvfrom)

Help on built-in function recvfrom:

recvfrom(...) method of socket.socket instance
    recvfrom(buffersize[, flags]) -> (data, address info)
    
    Like recv(buffersize, flags) but also return the sender's address info.



In [111]:
so.recvfrom(1024)

(b'hello\n', ('127.0.0.1', 51290))

In [114]:
class ChatServer:
    def __init__(self, ip='127.0.0.1', port=1234):
        self.addr = (ip, port)
        self.sock = socket.socket(type=socket.SOCK_DGRAM)
        self.event = threading.Event()
        self.clients = {}
    
    def recv(self):
        while not self.event.is_set():
            data, peer = self.sock.recvfrom(1024)
            data = data.decode().strip()
            self.clients[peer] = True

            for addr in self.clients.keys():
                self.sock.sendto('{}:{}\n{}'.format(peer[0], peer[1], data).encode(), addr)

    def start(self):
        # TODO 开启服务端
        # 是否需要启用多线程呢？
        self.sock.bind(self.addr)
        self.recv()
        
    def stop(self):
#         for so in self.clients.values():
#             so.close()
        self.sock.close()
        self.event.set()

In [115]:
chat_server = ChatServer()

In [116]:
chat_server.start()

OSError: [Errno 48] Address already in use

In [117]:
so.close()

In [118]:
chat_server.start()

KeyboardInterrupt: 

In [119]:
chat_server.stop()

In [120]:
import datetime

class ChatServer:
    def __init__(self, ip='127.0.0.1', port=1234):
        self.addr = (ip, port)
        self.sock = socket.socket(type=socket.SOCK_DGRAM)
        self.event = threading.Event()
        self.clients = {}
    
    def recv(self):
        while not self.event.is_set():
            now = datetime.datetime.now()
            data, peer = self.sock.recvfrom(1024)
            data = data.decode().strip()
            
            if data == '#ping#':
                self.clients[peer] = now
                continue
                
            disconnected = set()

            for addr, timestamp in self.clients.items():
                if (now - timestamp).total_seconds() > 10:
                    disconnected.add(addr)
                else:
                    self.sock.sendto('{}:{}\n{}'.format(peer[0], peer[1], data).encode(), addr)
                    
            for addr in disconnected:
                self.clients.pop(addr)

    def start(self):
        # TODO 开启服务端
        # 是否需要启用多线程呢？
        self.sock.bind(self.addr)
        self.recv()
        
    def stop(self):
#         for so in self.clients.values():
#             so.close()
        self.sock.close()
        self.event.set()

In [121]:
chat_server = ChatServer()

In [122]:
chat_server.start()

KeyboardInterrupt: 

## socketserver

In [123]:
import socketserver

In [124]:
class EchoHandler(socketserver.BaseRequestHandler):
    pass

In [128]:
class EchoHandler(socketserver.BaseRequestHandler):
    # 初始化工作
    def setup(self):
        super().setup()
    
    # 清理工作，通常情况下，不用重写
    def finish(self):
        super().finish()
    
    
    def handle(self):
        message = self.request.recv(1024).strip()
        # self.request == socket.socket().connect()
        self.request.send(message)

In [129]:
server = socketserver.ThreadingTCPServer(('127.0.0.1', 1234), EchoHandler)

try:
    server.serve_forever()
except KeyboardInterrupt:
    server.shutdown()

In [130]:
server.server_close()

In [131]:
class EchoHandler(socketserver.BaseRequestHandler):
    # 初始化工作
    def setup(self):
        print('setup {}:{}'.format(*self.request.getpeername()))
        super().setup()
    
    # 清理工作，通常情况下，不用重写
    def finish(self):
        print('finish {}:{}'.format(*self.request.getpeername()))
        super().finish()
    
    
    def handle(self):
        message = self.request.recv(1024).strip()
        # self.request == so, _ = socket.accept()
        self.request.send(message)

In [132]:
server = socketserver.ThreadingTCPServer(('127.0.0.1', 1234), EchoHandler)

try:
    server.serve_forever()
except KeyboardInterrupt:
    server.shutdown()
    server.server_close()

setup 127.0.0.1:57237
finish 127.0.0.1:57237


In [145]:
import datetime

class ChatHandler(socketserver.BaseRequestHandler):
    clients = {}
    
    def setup(self):
        self.event = threading.Event()
        self.clients[self.request.getpeername()] = self.request
        super().setup()
        
    def finish(self):
        del self.clients[self.request.getpeername()]
        self.event.set()
        super().finish()
        
    def handle(self):
        while not self.event.is_set():
            message = self.request.recv(1024).strip().decode()
            for so in self.clients.values():
                try:
                    so.send('{1}:{2}\n{0}'.format(message, *self.request.getpeername()).encode())
                except:
                    pass

In [147]:
server = socketserver.ThreadingTCPServer(('127.0.0.1', 5002), ChatHandler)

In [148]:
try:
    server.serve_forever()
except KeyboardInterrupt:
    server.server_close()
    server.shutdown()

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 57321)
Traceback (most recent call last):
  File "/Users/miracleYoung/.pyenv/versions/3.6.1/lib/python3.6/socketserver.py", line 639, in process_request_thread
    self.finish_request(request, client_address)
  File "/Users/miracleYoung/.pyenv/versions/3.6.1/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/miracleYoung/.pyenv/versions/3.6.1/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "<ipython-input-138-cd0fc36b21f4>", line 18, in handle
    message = self.request.recv(1024).strip().decode()
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
----------------------------------------


In [142]:
server.server_close()

In [149]:
help(socketserver.BaseServer)

Help on class BaseServer in module socketserver:

class BaseServer(builtins.object)
 |  Base class for server classes.
 |  
 |  Methods for the caller:
 |  
 |  - __init__(server_address, RequestHandlerClass)
 |  - serve_forever(poll_interval=0.5)
 |  - shutdown()
 |  - handle_request()  # if you do not use serve_forever()
 |  - fileno() -> int   # for selector
 |  
 |  Methods that may be overridden:
 |  
 |  - server_bind()
 |  - server_activate()
 |  - get_request() -> request, client_address
 |  - handle_timeout()
 |  - verify_request(request, client_address)
 |  - server_close()
 |  - process_request(request, client_address)
 |  - shutdown_request(request)
 |  - close_request(request)
 |  - service_actions()
 |  - handle_error()
 |  
 |  Methods for derived classes:
 |  
 |  - finish_request(request, client_address)
 |  
 |  Class variables that may be overridden by derived classes or
 |  instances:
 |  
 |  - timeout
 |  - address_family
 |  - socket_type
 |  - allow_reuse_addres